# Principal Component Regression (PCR)

This notebook implements **Principal Component Regression (PCR)** from scratch and helps to visualise the difference from the **Ordinary Least Square(OLS) Linear Regression**.

# Defining Principal Component Regression (PCR)

Going by the definition, **Principal Component Regression (PCR)** is an algorithm for **reducing the multi-collinearity of datatsets.**

# Problems with using multi-variate Linear Regression on these datasets

### High Variance Issue

In multi-variate Linear Regression, it may seem that the accuracy is very high on the training set and hence we have trained a brilliant model. But in reality, our model have overfit the training set and there is a high variance present.

Such a model is likely to perform extremely worse on predictions with new examples and have a very low test accuracy.

# Benefits of using PCR on these datasets

Applying **PCR** on such multi-collinear datasets has the following benefits:

**PCR** adds a slight bias to the dataset and reduces the degree of overfitting.

**PCR** is basically applying **PCA** to the dataset to obtain the **Principal Components (PCs)**. Then **Ordinary Least Square Linear Regression** is done on these obtain **Principal Components (PCs)**.

# Functioning of PCR

In **PCR**, instead of regressing the dependent variable on the independant variables directly, the principal components of the independant variables are used.

We apply **PCA** to the dataset to obtain the **Principal Components (PCs)**. **PCA** is a **Dimensionality Reduction Technique**. It combines two or more features to develop new features called **Principal Components (PCs)**. It combines features such that they can suite the target variable in a suitable way.

# PCA: Is it a Feature Selection Technique ?

People often confuse **PCA** as a **Feature Selection Technique**.

**PCA is not a Feature Selection Technique**. PCA doesn't select a set of fewer features from all those available in order to reduce the number of features. 

**PCA actually combines all the features to develop Principal Components (PCs), which are entirely different from the actual features.** 

These **Principal Components (PCs)** suite the target variable with high accuracy.

# Let's now implement PCR and OLS Linear Regression

We will implement both **PCR** and **Ordinary Least Square(OLS) Limear Regression** on the given dataset and will perform a quantitative comparision.

## Import libraries

First of all we need to import all the necessary libraries. We will be using the following libraries:

    **numpy**: